# InfluxDB access

https://influxdb-client.readthedocs.io/en/latest/index.html


In [ ]:
sessionId = "89db51de-22a6-4033-8201-2fc37a5fe905"

In [19]:
import os
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS

bucket = "racing"
org = "b4mad"
token = os.environ.get("INFLUXDB_TOKEN")
url = "https://telemetry.b4mad.racing/"

client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)

query_api = client.query_api()

query = f"""
from(bucket: "{bucket}") \
  |> range(start: 2022-05-08T11:43:37.251Z, stop:2022-05-08T12:30:39.231Z)
  |> filter(fn: (r) => r["_measurement"] == "laps")
  |> filter(fn: (r) => r["SessionId"] == "{sessionId}")
  |> filter(fn: (r) => (r["_field"] == "Brake" or r["_field"] == "Clutch" or r["_field"] == "Throttle"))
  |> aggregateWindow(every: 2s, fn: mean, createEmpty: false)
  |> yield(name: "mean")
"""

result = query_api.query(org=org, query=query)

results = []
for table in result:
    for record in table.records:
        results.append((record.get_field(), record.get_value()))

# print(results)

In [22]:
# show measurements

query = f"""
import "influxdata/influxdb/schema"

schema.measurements(bucket: "{bucket}")
"""


tables = query_api.query(org=org, query=query)
print("\n".join([f"{record['_value']}" for records in tables for record in records]))

laps


In [28]:
# show CarModel

query = f"""
import "influxdata/influxdb/schema"

schema.tagValues(bucket: "{bucket}", tag: "CarModel")
"""


tables = query_api.query(org=org, query=query)
print("\n".join([f"{record['_value']}" for records in tables for record in records]))

Audi TT Quattro '04
Audi TT RS (VLN)
Chevrolet Corvette C3
Formula Reiza
Formula Trainer
Formula Ultimate Gen2


In [27]:
# show SessionId

query = f"""
import "influxdata/influxdb/schema"

schema.tagValues(bucket: "{bucket}", tag: "SessionId")
"""


tables = query_api.query(org=org, query=query)
print("\n".join([f"{record['_value']}" for records in tables for record in records]))

0b9a2ac4-31e7-4bfb-a2e2-1e9aaf3eb6c0
0fa4e699-55cd-437b-bcc6-c73aae33d040
15178692-36c0-4be0-8f35-5085eb92d4bc
200078a6-3456-4760-9169-b51a4cc2a3f3
24312f23-5f0c-451c-a70b-fbbf056f55a1
332cb8d4-1756-4bc0-a038-c29c228a65bb
5fcb763a-a8a5-4f98-be67-8994d48e718d
71215aa7-4868-4df5-ac2a-1821f0f9dba1
77fdf557-1772-475f-88bc-8b244276318d
8babe735-5484-4a31-bcbf-01b0dd8e791e
8cf564ca-4b1e-4982-ad5a-f6c7a6ebe587
90ba2fbd-f2fd-4491-a9e0-8921f9e7dc08
98d6f049-9c5c-4c7c-b387-80bf528ad593
a95e6deb-1f01-46df-b915-ae603fc45709
b7406416-bb4c-4eea-a948-7b91633b83fc
bf35526c-14e5-4a0f-89db-f1176e4ea2da
cb891d95-929b-49d1-8e4c-f4b9271ed05e
e471ce1a-c56b-467e-9e61-977149ac3ed2
e65cbf77-b938-4f92-859f-1f447e238bf0
eddf481a-cfd4-44a2-b200-f87b97ab31d4


In [44]:
# show sessions for given track / car

query = f"""
from(bucket: "{bucket}")
  |> range(start: -10d)
  |> filter(fn: (r) => r["_measurement"] == "laps")
  |> keep(columns: ["SessionId", "CarModel", "TrackCode"])
  |> unique(column: "SessionId")
"""

# print as JSON
tables = query_api.query(org=org, query=query)
print(tables.to_json(indent=5))

# now as csv
csv_iterator = query_api.query_csv(org=org, query=query)
output = csv_iterator.to_values()
print(output)

[
     {
          "result": "_result",
          "table": 0,
          "CarModel": "Audi TT Quattro '04",
          "SessionId": "eddf481a-cfd4-44a2-b200-f87b97ab31d4",
          "TrackCode": "spa"
     },
     {
          "result": "_result",
          "table": 1,
          "CarModel": "Audi TT RS (VLN)",
          "SessionId": "90ba2fbd-f2fd-4491-a9e0-8921f9e7dc08",
          "TrackCode": "ks_zandvoort"
     },
     {
          "result": "_result",
          "table": 2,
          "CarModel": "Chevrolet Corvette C3",
          "SessionId": "200078a6-3456-4760-9169-b51a4cc2a3f3",
          "TrackCode": "SpaFrancorchamps-Spa_Francorchamps_2020"
     },
     {
          "result": "_result",
          "table": 3,
          "CarModel": "Chevrolet Corvette C3",
          "SessionId": "77fdf557-1772-475f-88bc-8b244276318d",
          "TrackCode": "SpaFrancorchamps-Spa_Francorchamps_2020"
     },
     {
          "result": "_result",
          "table": 4,
          "CarModel": "Formula Reiza

In [45]:
# query for given session

query = f"""
from(bucket: "{bucket}")
  |> range(start: -30d, stop: now())
  |> filter(fn: (r) => r["_measurement"] == "laps")
  |> filter(fn: (r) => r["_field"] == "TrackPositionPercent" or r["_field"] == "Brake" or r["_field"] == "CurrentLap")
  |> filter(fn: (r) => r["SessionId"] == "e65cbf77-b938-4f92-859f-1f447e238bf0")
  |> pivot(rowKey: ["_time"], columnKey: ["_field"], valueColumn: "_value")
  |> sort(columns: ["_time"], desc: false)
  |> keep(columns: ["_time", "CarModel", "TrackCode", "Brake", "TrackPositionPercent", "CurrentLap"])
  |> fill(usePrevious: true, column: "CurrentLap")
  |> fill(usePrevious: true, column: "Brake")
"""

result = query_api.query_data_frame(org=org, query=query)

result

,result,table,_time,CarModel,TrackCode,Brake,CurrentLap,TrackPositionPercent
0,_result,0,2022-08-28 15:35:39.512000+00:00,Formula Trainer,Monza-Monza_1991,NaN,1.0,0.000000
1,_result,0,2022-08-28 15:35:39.693000+00:00,Formula Trainer,Monza-Monza_1991,100.000000,1.0,NaN
2,_result,0,2022-08-28 15:35:43.410000+00:00,Formula Trainer,Monza-Monza_1991,0.000000,1.0,NaN
3,_result,0,2022-08-28 15:35:43.910000+00:00,Formula Trainer,Monza-Monza_1991,14.695001,1.0,NaN
4,_result,0,2022-08-28 15:35:44.077000+00:00,Formula Trainer,Monza-Monza_1991,28.806672,1.0,NaN
...,...,...,...,...,...,...,...,...
3719,_result,0,2022-08-28 15:46:14.644000+00:00,Formula Trainer,Monza-Monza_1991,7.135004,5.0,0.475549
3720,_result,0,2022-08-28 15:46:14.810000+00:00,Formula Trainer,Monza-Monza_1991,0.000000,5.0,0.475554
3721,_result,0,2022-08-28 15:46:14.977000+00:00,Formula Trainer,Monza-Monza_1991,0.000000,5.0,0.475557
3722,_result,0,2022-08-28 15:46:15.144000+00:00,Formula Trainer,Monza-Monza_1991,0.000000,5.0,0.475558
